In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report,multilabel_confusion_matrix
import ast
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification


In [2]:
similarity_df = pd.read_excel('ontology-w2v-similarity.xlsx')
similarity_df

,class,ontologyClass,Single Terms,W/V Embedding Vector,Similarity Terms,Embedding Vector of The Similar Term (Top10),Similarity Score (Top10)
0,http://www.semanticweb.org/ontoOCD#Obsession,Obsession,['obsession'],"[array([ 2.894364 , -0.14777394, 0.7309893 ,...","[['theme', 'spike', 'trigger', 'compulsion', '...","[[array([ 3.4748003 , -1.0715984 , 0.9782445 ...","[[0.8881328105926514, 0.8308006525039673, 0.83..."
1,http://www.semanticweb.org/ontoOCD#Compulsion,Compulsion,['compulsion'],"[array([ 2.8103964 , 1.5850425 , 1.023927 ,...","[['ritual', 'rumination', 'avoidance', 'rumina...","[[array([ 2.2761664 , 1.2755692 , 0.04933324...","[[0.9159335494041443, 0.8380370736122131, 0.82..."
2,http://www.semanticweb.org/ontoOCD#Intrusive_Urge,Intrusive_Urge,"['intrusive', 'urge']","[array([ 3.1508932 , 0.28482673, 0.42759928,...","[['obsessive', 'disturbing', 'unwanted', 'inst...","[[array([ 2.42819810e+00, 1.04331839e+00, 2....","[[0.8707727789878845, 0.8541218638420105, 0.85..."
3,http://www.semanticweb.org/ontoOCD#intrusive_t...,intrusive_thoughts,"['intrusive', 'thoughts']","[array([ 3.1508932 , 0.28482673, 0.42759928,...","[['obsessive', 'disturbing', 'unwanted', 'inst...","[[array([ 2.42819810e+00, 1.04331839e+00, 2....","[[0.8707727789878845, 0.8541218638420105, 0.85..."
4,http://www.semanticweb.org/ontoOCD#Thought_App...,Thought_Appraisal,"['thought', 'appraisal']","[array([ 4.2852626 , 0.57388484, -0.23509467,...","[['thoughtsit', 'act', 'scary', 'thoughtsi', '...","[[array([ 1.19772300e-01, 4.39339057e-02, 5....","[[0.7917605638504028, 0.7715624570846558, 0.76..."
...,...,...,...,...,...,...,...
111,http://www.semanticweb.org/ontoOCD#Symmetry_Ob...,Symmetry_Obsession,"['symmetry', 'obsession']","[array([ 3.58202785e-01, 1.39081366e-02, 1.2...","[['excessive', 'parallel', 'excessively', 'hoa...","[[array([ 1.1738789 , 0.44103888, -0.007551 ...","[[0.8884093165397644, 0.8776735067367554, 0.86..."
112,http://www.semanticweb.org/ontoOCD#Symmetry_Th...,Symmetry_Thought,"['symmetry', 'thought']","[array([ 3.58202785e-01, 1.39081366e-02, 1.2...","[['excessive', 'parallel', 'excessively', 'hoa...","[[array([ 1.1738789 , 0.44103888, -0.007551 ...","[[0.8884093165397644, 0.8776735067367554, 0.86..."
113,http://www.semanticweb.org/ontoOCD#Temperament...,Temperamental_Risk_Factor,"['temperamental', 'risk', 'factor']","[array([ 0.04437939, 0.02185036, -0.00414398,...","[['libido', 'shock', 'ing', 'territoriesgabong...","[[array([ 0.38253063, 0.13956283, 0.03322442...","[[0.8343713879585266, 0.8236863017082214, 0.82..."
114,http://www.semanticweb.org/ontoOCD#Washing,Washing,['washing'],"[array([ 2.1505804 , 1.6025057 , -0.44540238,...","[['cleaning', 'wash', 'showering', 'washer', '...","[[array([ 2.1215768 , 1.2788702 , -0.87235546...","[[0.916360080242157, 0.8899528980255127, 0.849..."


In [3]:
similarity_dict = dict(zip(similarity_df['Single Terms'],similarity_df['Similarity Terms']))
similarity_dict

{"['obsession']": "[['theme', 'spike', 'trigger', 'compulsion', 'ocds', 'behaviour', 'fear', 'rumination', 'issue', 'concern']]",
 "['compulsion']": "[['ritual', 'rumination', 'avoidance', 'ruminate', 'behaviour', 'obsession', 'perform', 'resisting', 'pattern', 'ruminating']]",
 "['intrusive', 'urge']": "[['obsessive', 'disturbing', 'unwanted', 'instrusive', 'bombarded', 'fleeting', 'obtrusive', 'distressing', 'violent', 'pocd'], ['impulse', 'strong', 'wanting', 'image', 'unwanted', 'scary', 'desire', 'automatically', 'overwhelming', 'sensation']]",
 "['intrusive', 'thoughts']": "[['obsessive', 'disturbing', 'unwanted', 'instrusive', 'bombarded', 'fleeting', 'obtrusive', 'distressing', 'violent', 'pocd'], ['reche', 'labour-intensive', 'mezain', 'designs', 'ummit', 'times2', 'ouja', 'battleim', '87.85', 'malaprop']]",
 "['thought', 'appraisal']": "[['thoughtsit', 'act', 'scary', 'thoughtsi', 'image', 'toughts', 'impulse', 'urge', 'react', 'rather'], ['informing', 'sho', 'observed', 'spr

In [4]:
key_list = []
value_list = []
for key, value in similarity_dict.items():
    # Get the key without square brackets and quotes
    clean_key = key.strip("[]'")
    clean_key = clean_key.split(',')
    for x in clean_key:
        x = x.strip(" '")
        key_list.append(x)

    if value == '[]':
        value_list.append([])
        continue

    list_of_lists = ast.literal_eval(value) if value else []
    for y in list_of_lists:
        value_list.append(y)


In [5]:
similarity_dict = dict(zip(key_list,value_list))
similarity_dict

{'obsession': ['theme',
  'spike',
  'trigger',
  'compulsion',
  'ocds',
  'behaviour',
  'fear',
  'rumination',
  'issue',
  'concern'],
 'compulsion': ['ritual',
  'rumination',
  'avoidance',
  'ruminate',
  'behaviour',
  'obsession',
  'perform',
  'resisting',
  'pattern',
  'ruminating'],
 'intrusive': ['obsessive',
  'disturbing',
  'unwanted',
  'instrusive',
  'bombarded',
  'fleeting',
  'obtrusive',
  'distressing',
  'violent',
  'pocd'],
 'urge': ['impulse',
  'strong',
  'wanting',
  'image',
  'unwanted',
  'scary',
  'desire',
  'automatically',
  'overwhelming',
  'sensation'],
 'thoughts': ['reche',
  'labour-intensive',
  'mezain',
  'designs',
  'ummit',
  'times2',
  'ouja',
  'battleim',
  '87.85',
  'malaprop'],
 'thought': ['thoughtsit',
  'act',
  'scary',
  'thoughtsi',
  'image',
  'toughts',
  'impulse',
  'urge',
  'react',
  'rather'],
 'appraisal': ['informing',
  'sho',
  'observed',
  'spring',
  'completed',
  'op',
  'commonly',
  'authorised',
  '

In [6]:
data = pd.read_excel('posts_with_label_and_pref_and_expand1-2.xlsx')
data

,post,prefLabel,compulsion,obsession,yy
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination..."
...,...,...,...,...,...
1556,"Hi all,\nI think I know the answers before I e...","['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int..."
1557,"Hi folks,\nI'm really struggling at the mo. I'...","['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination..."
1558,Well hello everyone\nAs you can see one of my ...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination..."
1559,Well hello!\n\nIve been struggling with ocd in...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination..."


In [92]:
similarity_dict

{'obsession': ['theme',
  'spike',
  'trigger',
  'compulsion',
  'ocds',
  'behaviour',
  'fear',
  'rumination',
  'issue',
  'concern'],
 'compulsion': ['ritual',
  'rumination',
  'avoidance',
  'ruminate',
  'behaviour',
  'obsession',
  'perform',
  'resisting',
  'pattern',
  'ruminating'],
 'intrusive': ['obsessive',
  'disturbing',
  'unwanted',
  'instrusive',
  'bombarded',
  'fleeting',
  'obtrusive',
  'distressing',
  'violent',
  'pocd'],
 'urge': ['impulse',
  'strong',
  'wanting',
  'image',
  'unwanted',
  'scary',
  'desire',
  'automatically',
  'overwhelming',
  'sensation'],
 'thoughts': ['reche',
  'labour-intensive',
  'mezain',
  'designs',
  'ummit',
  'times2',
  'ouja',
  'battleim',
  '87.85',
  'malaprop'],
 'thought': ['thoughtsit',
  'act',
  'scary',
  'thoughtsi',
  'image',
  'toughts',
  'impulse',
  'urge',
  'react',
  'rather'],
 'appraisal': ['informing',
  'sho',
  'observed',
  'spring',
  'completed',
  'op',
  'commonly',
  'authorised',
  '

In [10]:
ontology_keywords = data['yy']
similar_word = []
for x in ontology_keywords:
    instance_list = []
    keys = x.split(' ')
    keys = [word.lower() for word in keys]
    keys = [word.rstrip(',') for word in keys]
    keys = set(keys)
    
    for key in keys:
        words = similarity_dict.get(key)
        if words is not None:
            instance_list.extend(words)
    similar_word.append(instance_list)
        
    

In [11]:
len(similar_word)

1561

In [13]:
data['similar_words'] = similar_word
data

,post,prefLabel,compulsion,obsession,yy,similar_words
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination...","[med, antidepressant, ssri, prozac, fluoxetine..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int...","[excessive, parallel, excessively, hoard, tapp..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination...","[excessive, parallel, excessively, hoard, tapp..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination...","[excessive, parallel, excessively, hoard, tapp..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination...","[scale, fatigue, hugely, fortunately, concentr..."
...,...,...,...,...,...,...
1556,"Hi all,\nI think I know the answers before I e...","['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int...","[med, antidepressant, ssri, prozac, fluoxetine..."
1557,"Hi folks,\nI'm really struggling at the mo. I'...","['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination...","[sexually, violent, incest, abuse, fantasy, ra..."
1558,Well hello everyone\nAs you can see one of my ...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination...","[sexually, violent, incest, abuse, fantasy, ra..."
1559,Well hello!\n\nIve been struggling with ocd in...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination...","[scale, fatigue, hugely, fortunately, concentr..."


In [15]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)
#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [16]:
data['post'] = data['post'].apply(process_text)

In [17]:
data

,post,prefLabel,compulsion,obsession,yy,similar_words
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination...","[med, antidepressant, ssri, prozac, fluoxetine..."
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int...","[excessive, parallel, excessively, hoard, tapp..."
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination...","[excessive, parallel, excessively, hoard, tapp..."
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination...","[excessive, parallel, excessively, hoard, tapp..."
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination...","[scale, fatigue, hugely, fortunately, concentr..."
...,...,...,...,...,...,...
1556,hi think know answer even begin new forum thou...,"['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int...","[med, antidepressant, ssri, prozac, fluoxetine..."
1557,hi folk really struggling mo still hospital de...,"['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination...","[sexually, violent, incest, abuse, fantasy, ra..."
1558,well hello everyone see one obsession cheating...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination...","[sexually, violent, incest, abuse, fantasy, ra..."
1559,well hello struggling ocd many form obsession ...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination...","[scale, fatigue, hugely, fortunately, concentr..."


In [19]:
data['post'][0]

'hi since december since recovering prozac since 18 24 depression diagnosed ocd january psych bumped higher dosage prozac 80mg great job wanted go making gain weight freaking ocd revolves around cheating like know recent obsession put quote around cuz question whether obsession real well let start beginning went fiancé friend place drink played card game fiance whole time except went bathroom remember everything night despite alcohol system came home continued drinking fiance eventually blacked ended dream cheating partner someone seen earlier night well really cheating taking guy hand placing odd place image head nothing else really remember waking realize dream think stirred went right back sleep even remember keep thinking dream actually happened fiance keep telling whole time nothing happened think day driving nut always question whether really ocd doc diagnosed make feel better everything true real appt psych tomorrow think going write script zoloft apparently ssri weight gain ssr

In [27]:
data['similar_words'] = data['similar_words'].apply(lambda x: " ".join(x))
data

,post,prefLabel,compulsion,obsession,yy,similar_words
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Obsession, Aggressive obsession, Contamination...",med antidepressant ssri prozac fluoxetine tabl...
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Compulsion, Intrusive thoughts, Aggressive int...",excessive parallel excessively hoard tapping h...
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Obsession, Aggressive obsession, Contamination...",scale fatigue hugely fortunately concentration...
...,...,...,...,...,...,...
1556,hi think know answer even begin new forum thou...,"['Medication', 'anxiety', 'anxiety', 'bathroom...",1,1,"Compulsion, Intrusive thoughts, Aggressive int...",med antidepressant ssri prozac fluoxetine tabl...
1557,hi folk really struggling mo still hospital de...,"['obsession', 'fear', 'guilt', 'urge']",0,1,"Obsession, Aggressive obsession, Contamination...",sexually violent incest abuse fantasy rape hom...
1558,well hello everyone see one obsession cheating...,"['obsession', 'interest', 'OCD']",0,1,"Obsession, Aggressive obsession, Contamination...",sexually violent incest abuse fantasy rape hom...
1559,well hello struggling ocd many form obsession ...,"['obsession', 'Suicidal', 'behavior', 'regret'...",0,1,"Obsession, Aggressive obsession, Contamination...",scale fatigue hugely fortunately concentration...


In [30]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(data, test_size=test_split, stratify=data[['compulsion', 'obsession']].values, )
test_df,val_df = train_test_split(data,test_size=0.5,stratify=data[['compulsion', 'obsession']].values)
# Splitting the test set further into validation and new test sets.
# val_df = test_df.sample(frac=0.5)
# test_df.drop(val_df.index, inplace=True)

print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in validation set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 1248
Number of rows in validation set: 781
Number of rows in test set: 780


In [32]:
train_x,train_y = train_df['similar_words'], train_df[['compulsion','obsession']]
test_x,test_y = test_df['similar_words'], test_df[['compulsion','obsession']]
val_x,val_y = val_df['similar_words'], val_df[['compulsion','obsession']]

In [35]:

# Load BERT tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

# Tokenize and encode the training and testing texts
train_encodings = tokenizer(train_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = tokenizer(test_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
val_encodings = tokenizer(val_x.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')

# Convert labels to TensorFlow tensors
train_labels = tf.convert_to_tensor(train_y, dtype=tf.int32)
test_labels = tf.convert_to_tensor(test_y, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_y, dtype=tf.int32)

# Convert Hugging Face tokenized inputs to numpy arrays
train_encodings = {key: np.array(value) for key, value in train_encodings.items()}
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}
val_encodings = {key: np.array(value) for key, value in val_encodings.items()}
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)


history = model.fit(
    train_encodings,
    train_labels,
    validation_data=(val_encodings, val_labels),
    epochs=3,
    batch_size=32
)
predictions = model.predict(test_encodings)
predicted_labels = (tf.sigmoid(predictions.logits) > 0.5).numpy()  # Threshold for multi-label classification

# Evaluation
accuracy = accuracy_score(test_y, predicted_labels)
confusion_matrix = multilabel_confusion_matrix(test_y, predicted_labels)
classification_rep = classification_report(test_y, predicted_labels)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:\n", confusion_matrix)
print("Classification Report:\n", classification_rep)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
39/39 [==============================] - 1954s 49s/step - loss: 0.5056 - accuracy: 0.6450 - val_loss: 0.3219 - val_accuracy: 0.9245
Epoch 2/3
39/39 [==============================] - 1916s 49s/step - loss: 0.2605 - accuracy: 0.9519 - val_loss: 0.1865 - val_accuracy: 0.9654
Epoch 3/3
25/25 [==============================] - 217s 8s/step
Accuracy: 0.8435897435897436
Confusion Matrix:
 [[[444  27]
  [  0 309]]

 [[ 96  89]
  [  6 589]]]
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96       309
           1       0.87      0.99      0.93       595

   micro avg       0.89      0.99      0.94       904
   macro avg       0.89      0.99      0.94       904
weighted avg       0.89      0.99      0.94       904
 samples avg       0.93      1.00      0.95       904



In [36]:
model.save_pretrained('hugging_face/similarity_model')


In [37]:
data.to_excel('post_with_ontology_similarity.xlsx')

In [39]:
predicted_labels = predicted_labels.astype('int')
predicted_labels

array([[1, 1],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]])

In [40]:
test_df

,post,prefLabel,compulsion,obsession,yy,similar_words
1243,people call arm let u mobilize try defeat sill...,"['compulsion', 'fear', 'site', 'process', 'OCD...",1,0,"Compulsion, Thought, morbid thoughts, Mental i...",sensation movement groin skin instant reaction...
792,sorry long message hope well rough week respon...,"['flatulence', 'hope', 'OCD', 'fear', 'obsessi...",0,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
146,hello fellow sufferer well know suffer intrusi...,"['Intrusive thoughts', 'Thought', 'symptom', '...",0,1,"Intrusive thoughts, Aggressive intrusive thoug...",excessive parallel excessively hoard tapping h...
1126,thought id raise topic often see post topic be...,"['Thought', 'compulsion', 'fear']",1,0,"Compulsion, Thought, morbid thoughts, Mental i...",excessive parallel excessively hoard tapping h...
907,forgive ramblings mind quite literally place r...,"['obsession', 'joy']",0,1,"Obsession, Aggressive obsession, Contamination...",sexually violent incest abuse fantasy rape hom...
...,...,...,...,...,...,...
1192,hey trying follow 4 step hard moment outward c...,"['compulsion', 'OCD']",1,0,"Compulsion, compulsion, Compulsive behavior",ritualistic strengthen behavior hoarding resul...
265,finished level holiday tomorrow best friend oc...,"['Intrusive thoughts', 'Thought', 'anxiety', '...",0,1,"Intrusive thoughts, Aggressive intrusive thoug...",excessive parallel excessively hoard tapping h...
325,hi dealing contamination ocd flare three month...,"['compulsion', 'fear', 'writing', 'anxiety', '...",1,0,"Compulsion, anxiety, castration anxiety, perfo...",ritual rumination avoidance ruminate behaviour...
767,curious whether people success erp found obses...,['obsession'],0,1,"Obsession, Aggressive obsession, Contamination...",sexually violent incest abuse fantasy rape hom...


In [41]:
test_df['compulsion_prediction'] = predicted_labels[:,0]
test_df['obsession_prediction'] = predicted_labels[:,1]
test_df

,post,prefLabel,compulsion,obsession,yy,similar_words,compulsion_prediction,obsession_prediction
1243,people call arm let u mobilize try defeat sill...,"['compulsion', 'fear', 'site', 'process', 'OCD...",1,0,"Compulsion, Thought, morbid thoughts, Mental i...",sensation movement groin skin instant reaction...,1,1
792,sorry long message hope well rough week respon...,"['flatulence', 'hope', 'OCD', 'fear', 'obsessi...",0,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...,0,1
146,hello fellow sufferer well know suffer intrusi...,"['Intrusive thoughts', 'Thought', 'symptom', '...",0,1,"Intrusive thoughts, Aggressive intrusive thoug...",excessive parallel excessively hoard tapping h...,0,1
1126,thought id raise topic often see post topic be...,"['Thought', 'compulsion', 'fear']",1,0,"Compulsion, Thought, morbid thoughts, Mental i...",excessive parallel excessively hoard tapping h...,1,1
907,forgive ramblings mind quite literally place r...,"['obsession', 'joy']",0,1,"Obsession, Aggressive obsession, Contamination...",sexually violent incest abuse fantasy rape hom...,0,1
...,...,...,...,...,...,...,...,...
1192,hey trying follow 4 step hard moment outward c...,"['compulsion', 'OCD']",1,0,"Compulsion, compulsion, Compulsive behavior",ritualistic strengthen behavior hoarding resul...,1,0
265,finished level holiday tomorrow best friend oc...,"['Intrusive thoughts', 'Thought', 'anxiety', '...",0,1,"Intrusive thoughts, Aggressive intrusive thoug...",excessive parallel excessively hoard tapping h...,0,1
325,hi dealing contamination ocd flare three month...,"['compulsion', 'fear', 'writing', 'anxiety', '...",1,0,"Compulsion, anxiety, castration anxiety, perfo...",ritual rumination avoidance ruminate behaviour...,1,0
767,curious whether people success erp found obses...,['obsession'],0,1,"Obsession, Aggressive obsession, Contamination...",sexually violent incest abuse fantasy rape hom...,0,1


In [42]:
test_df.to_excel('post_with_ontology_similarity_test.xlsx')

##### Test of the post part of the data. The above was tested with the similar words

In [47]:

test_x_post = test_df['post']
test_x_post

1243    people call arm let u mobilize try defeat sill...
792     sorry long message hope well rough week respon...
146     hello fellow sufferer well know suffer intrusi...
1126    thought id raise topic often see post topic be...
907     forgive ramblings mind quite literally place r...
                              ...                        
1192    hey trying follow 4 step hard moment outward c...
265     finished level holiday tomorrow best friend oc...
325     hi dealing contamination ocd flare three month...
767     curious whether people success erp found obses...
1514    one ocd quirk driving fear looking license pla...
Name: post, Length: 780, dtype: object

In [44]:
train_df

,post,prefLabel,compulsion,obsession,yy,similar_words
79,hi new site joined think 6 year old daughter o...,"['hope', 'symptom', 'washing hands', 'obsessio...",0,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
983,bad day compulsion decided erp planned walk to...,"['outside', 'OCD', 'checking', 'anxiety', 'anx...",1,0,"Compulsion, Thought, morbid thoughts, Mental i...",check counting avoiding locked repeatedly goog...
1359,hi guy iv reading 4 steos refers checking comp...,"['checking', 'compulsion']",1,0,"Compulsion, compulsion, Checking, Compulsive b...",ritualistic strengthen behavior hoarding resul...
685,hello everyone posted dropped particularly bad...,"['drop', 'compulsion', 'anxiety', 'anxiety', '...",1,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
1108,well next issue need address dependence benzod...,"['Intrusive thoughts', 'Thought', 'Medication']",0,1,"Intrusive thoughts, Aggressive intrusive thoug...",med antidepressant ssri prozac fluoxetine tabl...
...,...,...,...,...,...,...
1422,hi guy quite good lateley blocking bad thought...,"['writing', 'compulsion', 'Thought', 'OCD']",1,0,"Compulsion, Thought, morbid thoughts, Mental i...",excessive parallel excessively hoard tapping h...
1115,thinking earlier tend talk obsessive thought q...,"['Thought', 'talking', 'obsession']",0,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
1277,hello everyone posted long time still struggli...,"['hope', 'Rituals', 'singing', 'toilet', 'guil...",1,1,"Obsession, Aggressive obsession, Contamination...",excessive parallel excessively hoard tapping h...
750,hope okay post know trich separate disorder wo...,"['hope', 'OCD', 'interest', 'Intrusive thought...",0,1,"Intrusive thoughts, Aggressive intrusive thoug...",excessive parallel excessively hoard tapping h...


In [48]:
test_encodings = tokenizer(test_x_post.tolist(), truncation=True, padding=True, max_length=128, return_tensors='tf')
test_encodings = {key: np.array(value) for key, value in test_encodings.items()}

predictions = model.predict(test_encodings)
predicted_labels = (tf.sigmoid(predictions.logits) > 0.5).numpy()  # Threshold for multi-label classification

# Evaluation
accuracy = accuracy_score(test_y, predicted_labels)
confusion_matrix = multilabel_confusion_matrix(test_y, predicted_labels)
classification_rep = classification_report(test_y, predicted_labels)

25/25 [==============================] - 280s 11s/step


In [49]:
accuracy

0.2858974358974359

In [51]:
predicted_labels = predicted_labels.astype('int')
predicted_labels

array([[1, 1],
       [1, 1],
       [1, 1],
       ...,
       [1, 1],
       [1, 1],
       [1, 1]])

In [52]:
accuracy = accuracy_score(test_y, predicted_labels)
accuracy

0.2858974358974359